In [1]:
import pandas as pd
import numpy as np
from google.colab import files

uploaded = files.upload()
df = pd.read_csv(list(uploaded.keys())[0])
df.head()

Saving train.csv to train.csv


,Unnamed: 0,Name,Location,Year,Kilometers_Driven,Fuel_Type,Transmission,Owner_Type,Mileage,Engine,Power,Seats,New_Price,Price
0,1,Hyundai Creta 1.6 CRDi SX Option,Pune,2015,41000,Diesel,Manual,First,19.67 kmpl,1582 CC,126.2 bhp,5.0,NaN,12.50
1,2,Honda Jazz V,Chennai,2011,46000,Petrol,Manual,First,13 km/kg,1199 CC,88.7 bhp,5.0,8.61 Lakh,4.50
2,3,Maruti Ertiga VDI,Chennai,2012,87000,Diesel,Manual,First,20.77 kmpl,1248 CC,88.76 bhp,7.0,NaN,6.00
3,4,Audi A4 New 2.0 TDI Multitronic,Coimbatore,2013,40670,Diesel,Automatic,Second,15.2 kmpl,1968 CC,140.8 bhp,5.0,NaN,17.74
4,6,Nissan Micra Diesel XV,Jaipur,2013,86999,Diesel,Manual,First,23.08 kmpl,1461 CC,63.1 bhp,5.0,NaN,3.50


**A) Handling Missing Values**

I checked missing values in each column.
For numerical columns, I used the median, which is more robust to outliers.
For categorical columns, I used the mode, since dropping them could cause unnecessary data loss.

In [2]:
df.isna().sum()
for col in df.columns:
    if df[col].dtype != 'object':
        df[col].fillna(df[col].median(), inplace=True)
    else:
        df[col].fillna(df[col].mode()[0], inplace=True)

df.isna().sum()

/tmp/ipython-input-1055508139.py:4: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df[col].fillna(df[col].median(), inplace=True)
/tmp/ipython-input-1055508139.py:6: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try usin

,0
Unnamed: 0,0
Name,0
Location,0
Year,0
Kilometers_Driven,0
Fuel_Type,0
Transmission,0
Owner_Type,0
Mileage,0
Engine,0


**B) Removing Units**

Some columns contain values with units like:

*   "kmpl" in Mileage
*   "CC" in Engine
*   "bhp" in Power
*   "lakh" in New_price

These units were removed to convert the columns into pure numerical values.

In [6]:
import re

def clean_numeric(col):

    return col.astype(str).apply(lambda x: re.sub(r'[^0-9.]', '', x)).replace('', np.nan).astype(float)

if 'Mileage' in df.columns:
    df['Mileage'] = clean_numeric(df['Mileage'])

if 'Engine' in df.columns:
    df['Engine'] = clean_numeric(df['Engine'])

if 'Power' in df.columns:
    df['Power'] = clean_numeric(df['Power'])

if 'New_price' in df.columns:
    df['New_price'] = clean_numeric(df['New_price'])

df.head()



,Unnamed: 0,Name,Location,Year,Kilometers_Driven,Fuel_Type,Transmission,Owner_Type,Mileage,Engine,Power,Seats,New_Price,Price
0,1,Hyundai Creta 1.6 CRDi SX Option,Pune,2015,41000,Diesel,Manual,First,19.67,1582.0,126.20,5.0,4.78 Lakh,12.50
1,2,Honda Jazz V,Chennai,2011,46000,Petrol,Manual,First,13.00,1199.0,88.70,5.0,8.61 Lakh,4.50
2,3,Maruti Ertiga VDI,Chennai,2012,87000,Diesel,Manual,First,20.77,1248.0,88.76,7.0,4.78 Lakh,6.00
3,4,Audi A4 New 2.0 TDI Multitronic,Coimbatore,2013,40670,Diesel,Automatic,Second,15.20,1968.0,140.80,5.0,4.78 Lakh,17.74
4,6,Nissan Micra Diesel XV,Jaipur,2013,86999,Diesel,Manual,First,23.08,1461.0,63.10,5.0,4.78 Lakh,3.50


**C) One-Hot Encoding**

Categorical columns Fuel_Type and Transmission were converted into numerical variables using one-hot encoding.
*drop_first=True* was used to avoid dummy variable trap.

In [7]:
categorical = ['Fuel_Type', 'Transmission']
existing = [c for c in categorical if c in df.columns]

df = pd.get_dummies(df, columns=existing, drop_first=True)
df.head()

,Unnamed: 0,Name,Location,Year,Kilometers_Driven,Owner_Type,Mileage,Engine,Power,Seats,New_Price,Price,Fuel_Type_Electric,Fuel_Type_Petrol,Transmission_Manual
0,1,Hyundai Creta 1.6 CRDi SX Option,Pune,2015,41000,First,19.67,1582.0,126.20,5.0,4.78 Lakh,12.50,False,False,True
1,2,Honda Jazz V,Chennai,2011,46000,First,13.00,1199.0,88.70,5.0,8.61 Lakh,4.50,False,True,True
2,3,Maruti Ertiga VDI,Chennai,2012,87000,First,20.77,1248.0,88.76,7.0,4.78 Lakh,6.00,False,False,True
3,4,Audi A4 New 2.0 TDI Multitronic,Coimbatore,2013,40670,Second,15.20,1968.0,140.80,5.0,4.78 Lakh,17.74,False,False,False
4,6,Nissan Micra Diesel XV,Jaipur,2013,86999,First,23.08,1461.0,63.10,5.0,4.78 Lakh,3.50,False,False,True


**D) Creating a New Feature: Car Age**

A new column Car_Age was added, calculated as:

*Car_Age = Current_Year – Year*


This is done using Python's datetime module.

In [8]:
from datetime import datetime
current_year = datetime.now().year

if 'Year' in df.columns:
    df['Car_Age'] = current_year - df['Year']

df.head()


,Unnamed: 0,Name,Location,Year,Kilometers_Driven,Owner_Type,Mileage,Engine,Power,Seats,New_Price,Price,Fuel_Type_Electric,Fuel_Type_Petrol,Transmission_Manual,Car_Age
0,1,Hyundai Creta 1.6 CRDi SX Option,Pune,2015,41000,First,19.67,1582.0,126.20,5.0,4.78 Lakh,12.50,False,False,True,10
1,2,Honda Jazz V,Chennai,2011,46000,First,13.00,1199.0,88.70,5.0,8.61 Lakh,4.50,False,True,True,14
2,3,Maruti Ertiga VDI,Chennai,2012,87000,First,20.77,1248.0,88.76,7.0,4.78 Lakh,6.00,False,False,True,13
3,4,Audi A4 New 2.0 TDI Multitronic,Coimbatore,2013,40670,Second,15.20,1968.0,140.80,5.0,4.78 Lakh,17.74,False,False,False,12
4,6,Nissan Micra Diesel XV,Jaipur,2013,86999,First,23.08,1461.0,63.10,5.0,4.78 Lakh,3.50,False,False,True,12


**E) Data Manipulation**

Performed the following operations:

*   Select: choosing specific columns

*   Filter: filtering rows based on a condition

*   Mutate: creating a calculated column
*   Arrange: sorting data


*   Summarize + Group_by: summary statistics grouped by a categorical variable

These operations help understand and analyze cleaned data effectively.

In [9]:
selected_df = df[['Year', 'Mileage', 'Engine', 'Price']] if 'Price' in df.columns else df.head()

filtered_df = df[df['Mileage'] > 15] if 'Mileage' in df.columns else df.head()

renamed_df = df.rename(columns={'Engine': 'Engine_CC'}) if 'Engine' in df.columns else df.head()

if 'Mileage' in df.columns and 'Engine' in df.columns:
    df['Mileage_per_CC'] = df['Mileage'] / df['Engine']
sorted_df = df.sort_values(by='Price', ascending=False) if 'Price' in df.columns else df.head()

if 'Fuel_Type_Petrol' in df.columns:
    summary = df.groupby('Fuel_Type_Petrol').agg({'Price': ['mean', 'max', 'min']})
else:
    summary = df.head()

summary

Price             
                       mean    max   min
Fuel_Type_Petrol                        
False             12.960632  160.0  0.60
True               5.756688  120.0  0.44